Missionaries and Cannibals

* There are N missionaries and M cannibals on the left bank of a river, and N => M
* There is a boat on the left bank of the river
* The goal is to move all missionaries and all cannibals to the right bank
* The boat can only move if there is at least one person in it
* The boat can hold at most two people
* The number of missionaries must never be less than the number of cannibals on either the left or right bank


In [1]:
from searchFramework import aStarSearch
from searchClientInterface import WorldState
import copy

class MCWorldState(WorldState):

    def __init__(self, ml, cl):
        self._state = {"ml": ml, "cl": cl, "mr": 0, "cr": 0, "mb": 0, "cb": 0, "bp": "l"}
    
    def val(self, elt):
        return self._state[elt]
    
    def increment(self, elt):
        self._state[elt] += 1
    
    def decrement(self, elt):
        self._state[elt] -=1
    
    def __str__(self):
        return "{" + str(self._state) + "}"
    
    def __eq__(self, other):
        if isinstance(other, MCWorldState):
            return self._state == other._state
        else:
            return False

    def __hash__(self):
        return hash(str(self._state))
    
    def successors(self):
        candidates = (self.em(), self.ec(), self.dm(), self.dc(), self.mb())
        return [c for c in candidates if c] 
    
    def em(self):
        if self.val("bp") == "l":
            if self.val("ml") > 0 and self.val("ml") > self.val("cl") and self.val("mb") + self.val("cb") < 2:
                s = copy.deepcopy(self)
                s.decrement("ml")
                s.increment("mb")
                return (s, "em")
        elif self.val("bp") == 'r':
            if self.val("mr") > 0 and self.val("mr") > self.val("cr") and self.val("mb") + self.val("cb") < 2:
                s = copy.deepcopy(self)
                s.decrement("mr")
                s.increment("mb")
                return (s, "em")
        else:
            raise Exception(f"Invalid boat state {self.val('bp')}")
        return None
    
    def ec(self):
        if self.val("bp") == "l":
            if self.val("cl") > 0 and self.val("mb") + self.val("cb") < 2:
                s = copy.deepcopy(self)
                s.decrement("cl")
                s.increment("cb")
                return (s, "ec")
        elif self.val("bp") == "r":
            if self.val("cr") > 0 and self.val("mb") + self.val("cb") < 2:
                s = copy.deepcopy(self)
                s.decrement("cr")
                s.increment("cb")
                return (s, "ec")
        else:
            raise Exception(f"Invalid boat state {self.val('bp')}")
        return None
            
    def dm(self):
        if self.val("bp") == "l":
            if self.val("mb") > 0:
                s = copy.deepcopy(self)
                s.decrement("mb")
                s.increment("ml")
                return (s, "dm")
        elif self.val("bp") == "r":
                if self.val("mb") > 0 :
                    s = copy.deepcopy(self)
                    s.decrement("mb")
                    s.increment("mr")
                    return (s, "dm")
        else:
            raise Exception(f"Invalid boat state {self.val('bp')}")
        return None
    
    def dc(self):
        if self.val("bp") == "l":
            if self.val("cb") > 0 and self.val("ml") > self.val("cl"):
                s = copy.deepcopy(self)
                s.decrement("cb")
                s.increment("cl")
                return (s, "dc")
        elif self.val("bp") == "r":
            if self.val("cb") > 0 and self.val("mr") > self.val("cr"):
                s = copy.deepcopy(self)
                s.decrement("cb")
                s.increment("cr")
                return (s, "dc")
        else:
            raise Exception(f"Invalid boat state {self.val('bp')}") 
        return None
            
    def mb(self):
        if self.val("mb") + self.val("cb") > 0:
            s = copy.deepcopy(self)
            s._state["bp"]  = 'l' if s._state['bp'] == 'r' else 'r'
            return (s, "mb")
        return None

In [2]:
from searchClientInterface import Problem

class MCProblem(Problem):
    def __init__(self, ml, cl):
        self._state = MCWorldState(ml, cl)
 
    def initial(self):
        return self._state
    
    def isGoal(self, state):
        return state.val('ml') + state.val('cl') + state.val('mb') + state.val('cb') == 0

In [3]:
from searchClientInterface import Evaluator
def bfsGuesser(state):
    return 0

def bfsCoster(actions):
    return len(actions)

def bfsEvaluator():
    return Evaluator(bfsCoster, bfsGuesser)

In [4]:
def dfsGuesser(state):
    return 0

def dfsCoster(actions):
    return -len(actions)

def dfsEvaluator():
    return Evaluator(dfsCoster, dfsGuesser)

In [5]:
from searchFramework import aStarSearch

In [6]:
print(aStarSearch(MCProblem(1,1), bfsEvaluator()))

(['ec', 'em', 'mb', 'dm', 'dc'], (0.0, 12, 5, 4))


In [7]:
print(aStarSearch(MCProblem(3,3), bfsEvaluator()))

(['ec', 'em', 'mb', 'dm', 'mb', 'ec', 'mb', 'dc', 'mb', 'em', 'mb', 'dm', 'mb', 'ec', 'mb', 'dc', 'mb', 'em', 'mb', 'dm', 'dc'], (0.0, 65, 31, 6))


In [8]:
soln, stats = aStarSearch(MCProblem(10000, 10000), bfsEvaluator())
print(stats)

(40.359375, 279981, 139989, 6)


In [9]:
soln, stats = aStarSearch(MCProblem(10000, 10000), dfsEvaluator(), None, 50000)
print(stats)

(105.09375, 50001, 19443, 16670)


In [11]:
def mcGuesser(state):
    return state.val('ml') * 3 + state.val('cl') * 3 + state.val('mb') + state.val('cb')
def mcEvaluator():
    return Evaluator(lambda actions: len(actions), mcGuesser)

In [13]:
soln, stats = aStarSearch(MCProblem(10000, 10000), mcEvaluator())
print(stats)

(35.953125, 279968, 139980, 14)
